# ViralCutter
Uma alternativa gratuita ao `opus.pro` e ao `vidyo.ai`

# Suporte em:
[![](https://dcbadge.limes.pink/api/server/tAdPHFAbud)](https://discord.gg/tAdPHFAbud)

In [ ]:
#@title 🎬 VIRALCUTTER COLAB (VRAM Fix + Drive Sync)
#@markdown Este script roda o ViralCutter com otimização de memória e sincroniza os vídeos processados com seu Google Drive.

import os
import shutil
import subprocess
import threading
import time
import sys
from IPython.display import clear_output
from google.colab import drive

# ==========================================
# ⚙️ CONFIGURAÇÕES (EDITE AQUI)
# ==========================================
PASTA_DRIVE_FINAL = "/content/drive/MyDrive/ViralCutter_Prontos" # Onde salvar

# ==========================================
# 1. MONTAGEM DO DRIVE (NATIVO COLAB)
# ==========================================
print("1️⃣ Conectando ao Google Drive...")
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

os.makedirs(PASTA_DRIVE_FINAL, exist_ok=True)
print(f"✅ Drive conectado! Vídeos irão para: {PASTA_DRIVE_FINAL}")

# ==========================================
# 2. INSTALAÇÃO OTIMIZADA (VRAM FIX)
# ==========================================
# Verifica se já está instalado para economizar tempo
if not os.path.exists("/content/ViralCutter"):
    print("\n📦 Instalando ViralCutter Otimizado (3-5 min)...")

    # Clone
    subprocess.run("git clone https://github.com/RafaelGodoyEbert/ViralCutter.git /content/ViralCutter", shell=True)
    os.chdir("/content/ViralCutter")

    # Instalação do gerenciador rápido (UV)
    subprocess.run("pip install uv -q", shell=True)

    # Drivers de Sistema (FFmpeg e CUDA basics)
    subprocess.run("apt-get update -y -qq && apt-get install -y libcudnn8 ffmpeg xvfb -qq", shell=True)

    # Cria ambiente
    subprocess.run(["uv", "venv", ".venv"], check=True)

    # --- OTIMIZAÇÃO DE VRAM AQUI ---
    # Instalamos o faster-whisper e forçamos versões específicas do Torch
    # para evitar que o Colab carregue CUDA kernels duplicados.
    print("   🚀 Otimizando dependências de Vídeo e IA...")

    cmds = [
        # Motor de transcrição leve
        "uv pip install --python .venv faster-whisper",
        # Bibliotecas base
        "uv pip install --python .venv git+https://github.com/m-bain/whisperx.git",
        "uv pip install --python .venv -r requirements.txt",
        "uv pip install --python .venv -U --pre 'yt-dlp[default]'",
        # Gemini e Processamento
        "uv pip install --python .venv google-generativeai pandas onnxruntime-gpu",
        # Correção de versão do Transformers (Evita erro de alinhamento)
        "uv pip install --python .venv transformers==4.46.3 accelerate>=0.26.0",
        # Torch Otimizado para T4 (Colab)
        "uv pip install --python .venv torch==2.3.1+cu121 torchvision==0.18.1+cu121 torchaudio==2.3.1+cu121 --index-url https://download.pytorch.org/whl/cu121",
        # Visão Computacional (MediaPipe leve)
        "uv pip install --python .venv insightface",
        "uv pip uninstall --python .venv mediapipe protobuf flatbuffers",
        "uv pip install --python .venv 'mediapipe>=0.10.0' 'protobuf>=3.20,<5.0' 'flatbuffers>=2.0'"
    ]

    for cmd in cmds:
        subprocess.run(cmd, shell=True, check=True)

    print("✅ Instalação Concluída!")
else:
    os.chdir("/content/ViralCutter")
    print("\n✅ Sistema já instalado. Pulando etapa.")

# ==========================================
# 3. MONITOR AUTOMÁTICO (ENGINE)
# ==========================================
def engine_loop():
    print("👀 Engine: Monitorando pasta VIRALS...")
    processed = set()
    WATCH_DIR = "/content/ViralCutter/VIRALS"

    while True:
        if os.path.exists(WATCH_DIR):
            for r, d, f in os.walk(WATCH_DIR):
                for file in f:
                    if not file.endswith(".mp4"): continue
                    # Ignora arquivos temporários e originais
                    if any(x in file for x in ["input", "temp", "original"]): continue

                    full_path = os.path.join(r, file)

                    # Verifica se o arquivo terminou de ser salvo
                    if full_path not in processed and os.path.getsize(full_path) > 1e6:
                        s1 = os.path.getsize(full_path)
                        time.sleep(5)
                        if os.path.getsize(full_path) != s1: continue

                        print(f"\n💎 NOVO VÍDEO DETECTADO: {file}")

                        # Envia para o Drive
                        print(f"   ☁️ Salvando no Drive: {PASTA_DRIVE_FINAL}...")
                        shutil.copy(full_path, os.path.join(PASTA_DRIVE_FINAL, os.path.basename(full_path)))
                        print("   ✅ Concluído!")

                        processed.add(full_path)

        time.sleep(5)

# Inicia o monitor em segundo plano
threading.Thread(target=engine_loop, daemon=True).start()

# ==========================================
# 4. INICIAR INTERFACE
# ==========================================
print("\n🚀 INICIANDO INTERFACE...")
print("⚠️ CLIQUE NO LINK PÚBLICO (gradio.live) ABAIXO:")
print("="*60)

# Configuração de Display Virtual (Necessário para Colab)
subprocess.Popen(['Xvfb', ':1', '-screen', '0', '2560x1440x8'], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
time.sleep(2)
os.environ.update({'DISPLAY':':1.0', 'MPLBACKEND':'Agg', 'CUDA_VISIBLE_DEVICES':'0'})

# Roda o App
!/content/ViralCutter/.venv/bin/python webui/app.py --colab

#Créditos

Inspirado no [reels clips automator](https://github.com/eddieoz/reels-clips-automator) e no [YoutubeVideoToAIPoweredShorts](https://github.com/Fitsbit/YoutubeVideoToAIPoweredShorts)<br>

---
![Rafa.png](https://i.imgur.com/cGknQpU.png;base64)

Desenvolvido por **Rafa.Godoy**<br>
[ ![GitHub](https://img.shields.io/badge/github-%23121011.svg?style=for-the-badge&logo=github&logoColor=white) ](https://github.com/rafaelGodoyEbert)<br>
[ ![X](https://img.shields.io/twitter/url?url=https%3A%2F%2Ftwitter.com%2FGodoyEbert) ](https://twitter.com/GodoyEbert)<br>
[Instagram](https://www.instagram.com/rafael.godoy.ebert/)<br>
[ ![](https://dcbadge.vercel.app/api/server/aihubbrasil) ](https://discord.gg/aihubbrasil)

`0.8v Alpha`<br>

Apenas uma alternativa gratuita ao `opus.pro` e ao `vidyo.ai`<br>
